# Ev3devsim_nb

Exploring ??? in a notebook context.

In [ ]:
import jp_proxy_widget
from jp_proxy_widget import js_context
from IPython.display import HTML, display

from IPython import get_ipython
from datetime import datetime

In [ ]:
from pkg_resources import resource_exists, resource_filename

def get_file_path(fn):
    """Get file content from local store."""
    # This should work locally or in package
    try:
        resource_exists(__name__, fn)
        return resource_filename(__name__, fn)
    except:
        pass
    return fn




In [ ]:
import os
import os.path
import shutil

In [ ]:
# Copy over background files
_dir = "backgrounds"

In [ ]:
#Py 3.8?
#shutil.copytree(get_file_path(_dir), _dir, dirs_exist_ok=True)
if not os.path.isdir(_dir):
    os.makedirs(_dir)
_path = get_file_path(_dir)
for f in os.listdir(_path):
    shutil.copy(os.path.join(get_file_path(_dir), f), _dir)

In [ ]:
with open(get_file_path('templates/studio.html'), 'r') as f:
    html = f.read()

In [ ]:
with open(get_file_path('templates/studio.js'), 'r') as f:
    script_built = f.read()

In [ ]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def sim_magic(line, cell):
    "Send code to simulator."
    eval(line).set_element("prog", cell)

In [ ]:
prog='''
# Demo of a simple proportional line follower using two sensors
# It's deliberately flawed and will exit with errors in some circumstances;
# try fixing it!

from ev3dev2.motor import MoveSteering, OUTPUT_B, OUTPUT_C
from ev3dev2.sensor import INPUT_1, INPUT_2, INPUT_3, INPUT_4
from ev3dev2.sensor.lego import ColorSensor, GyroSensor, UltrasonicSensor

steering_drive = MoveSteering(OUTPUT_B, OUTPUT_C)

colorLeft = ColorSensor(INPUT_2)
colorRight = ColorSensor(INPUT_3)
gyro = GyroSensor(INPUT_4)
ultrasonic = UltrasonicSensor(INPUT_1)

GAIN = 0.5

while True:
    print('Gyro: ' + str(gyro.angle_and_rate))
    print('Ultrasonic: ' + str(ultrasonic.distance_centimeters))
    print('Color: ' + str(colorRight.color))
    error = colorLeft.reflected_light_intensity - colorRight.reflected_light_intensity
    correction = error * GAIN
    steering_drive.on(correction, 20)
        
'''

In [ ]:
#myprint = Print_Repr()

In [ ]:
class Ev3DevWidget(jp_proxy_widget.JSProxyWidget):
    """Class to define an Ev3DevSim IPywidget."""
    def __init__(self, *pargs, **kwargs):
        super(Ev3DevWidget, self).__init__(*pargs, **kwargs)
        e = self.element

        # Call some standard jQuery method on the widget element:
        e.empty()
        # This is for the controls as well as the simulator
        e.width("1000") # 1181px 2362
        e.height("1000") # 551px 1143
        # self.require_js("ace", "ace-src-min/ace.js")
        self.load_js_files([get_file_path("js/skulpt.min.js"), get_file_path("js/skulpt-stdlib.js"),
                            get_file_path("js/EV3devSim.js"), get_file_path("js/plotly.min.js")])
        self.load_css(get_file_path("css/main.css"))
        e.html(html)
        #self.require_js("saveAs", "js/FileSaver.js")

        self.js_init("ready();", ready=self.ready)
        
        self.results_log = []
        self.count = 0
    
    
    def clear_datalog(self):
        """Clear the datalog in the simulator ipywidget object."""
        self.results_log = []
        self.count = 0
        
    def print_repr(self, obj):
        """Callback function when anything is written to the simulator print window."""
        # TO DO
        # - log all sensor channels
        # - add timestamp
        if obj.startswith('Ultrasonic') or obj.startswith('Colour'):
            typ = obj.split(': ')[0]
            val = float(obj.split(': ')[1])
            self.results_log.append({'index': datetime.utcnow(), typ: val})
            self.count += 1

    def ready(self):
        "Initialise the simulator."
        # The report_callback function is called from the simulator code
        # whenever something is written to the output window.
        # At the moment, we use this as a basis for data logging.
        #self.js_init(script_built, report_callback=print_repr)
        self.js_init(script_built, report_callback=self.print_repr)

In [ ]:
class SimInformation:
    data = "unknown: apparently the callback hasn't been called yet."
    
def sim_callback(data):
    """Set SimInformation class data value."""
    SimInformation.data = data

def sim_get_data(sim_widget, sim_var="sim.robotStates"):
    """Run async data request on simulator widget."""
    sim_widget.get_value_async(sim_callback, sim_var)

def sim_report(retval=False):
    """Get simulator data value."""
    print("Python thinks the sim data is: " + repr(SimInformation.data))
    if retval:
        return SimInformation.data

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
testEmbed = Ev3DevWidget()

In [ ]:
sim_report()

In [ ]:
asyncio.run(sim_get_data(testEmbed))
sim_report()

# sim_report(testEmbed)

In [ ]:
%%sim_magic testEmbed
print("lsdhfkjfsadhfkjh")

In [ ]:

#prog='print("hello world")'
#testEmbed.set_element("prog", "print('hello')")
testEmbed.set_element("prog", prog)
display(testEmbed)

In [ ]:
display(testEmbed)

In [ ]:
import holoviews as hv
import numpy as np

from holoviews.streams import Pipe, Buffer

hv.extension('bokeh')

dfbuffer = Buffer(np.zeros((0, 2)), length=20)

In [ ]:
dfbuffer

In [ ]:
#from holoviews.streams import Pipe, Buffer

#hv.extension('bokeh')

#dfbuffer = Buffer(np.zeros((0, 2)), length=20)

z = []
zz = []
count=0
def print_repr(obj):
    """Display object"""
    #global dfbuffer
    global count
    if obj.startswith('Ultrasonic'):
        typ = obj.split(': ')[0]
        val = float(obj.split(': ')[1])
        z.append((typ, val))
        #dfbuffer.send(np.array([[count, val]]))
        count += 1
            #zz.append(dfbuffer)
            #zz.append(np.array([[count, val]]))
            #zz.append((typ, val))
            #dfbuffer.send(np.array([[count, val]]))
            #count += 1

In [ ]:
hv.DynamicMap(hv.Curve, streams=[dfbuffer]).opts(padding=0.1, width=600, color = 'green',)

In [ ]:
# Evaluate the Window location in the Javascript context and call back to Python
testEmbed.get_value_async(sim_callback, "sim.robotStates")

In [ ]:
print("Python thinks the sim data is: " + repr(SimInformation.data))

In [ ]:
testEmbed.get_value_async(sim_callback, "sim")


In [ ]:
print("Python thinks the prog is: " + repr(SimInformation.data))